# Dependencies

In [107]:
import pandas as pd
import numpy as np
from pathlib import Path
import gurobipy as gp

In [108]:
data_path = Path("../clean_data/48_Industry_Portfolios.CSV")
df = pd.read_csv(data_path, index_col=0)
df.index = pd.to_datetime(df.index, format="%Y%m")  # clean the index to be datetime

# constants
start_date = "2019-12-01"
N = 48
df = df.loc[df.index >= start_date, :]  # select last 5 years

min(df.var())

17.5711993220339

In [1]:
# QUESTION 1
cov = df.cov()
mean = df.mean(axis=0)
std = np.sqrt(np.diag(cov))
ret = 4
K = 3
l = 0.005 #min position size

# Create an empty optimization model
m = gp.Model()

# Add variables: x[i] denotes the proportion of capital invested in stock i
# 0 <= x[i] <= 1
x = m.addMVar(len(mean), lb=-np.inf, ub=np.inf, name="x")
b = m.addMVar(len(mean), vtype=gp.GRB.BINARY, name="b")

# Budget constraint: all investments sum up to 1
m.addConstr(x.sum() == 1, name="Budget_Constraint")
m.addConstr(-b < x < b, name="Indicator")
m.addConstr(b.sum() <= K, name="Concentration")
m.addConstr(mean.to_numpy() @ x >= ret , name="Target_Return")

# Define objective function: Minimize overall risk
m.setObjective(x.T @ cov.to_numpy() @ x, gp.GRB.MINIMIZE)

NameError: name 'df' is not defined

In [104]:
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 3800X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 195 rows, 144 columns and 528 nonzeros
Model fingerprint: 0x69626afd
Model has 1176 quadratic objective terms
Variable types: 96 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+100]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 144 rows and 48 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, 

In [ ]:
print(f"Minimum risk:     {m.ObjVal:.6f}")
print(f"Expected return:  {mean @ x.X:.6f}")
print(f"Solution time:    {m.Runtime:.2f} seconds\n")

# Print investments (with non-negligible value, i.e., > 1e-5)
positions = pd.Series(name="Position", data=x.X, index=mean.index)
print(f"Number of assets: {positions[abs(positions) > 1e-5].count()}\n")
print(positions[abs(positions) > 1e-5])

AttributeError: Unable to retrieve attribute 'ObjVal'